In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()#'vice')

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [ ]:
collect_results = crawler.collect(); display(*collect_results)

9.59% (16.71 seconds): Collecting URLs... investing.com       

In [25]:
collect_results[2]['thinkprogress']

82

# 2. 모은 URL중 이미 받은 것은 버린다

In [26]:
select_results = crawler.select(); display(*select_results)

100.00% (2.22 seconds): Selecting URLs... cryptonews          

n_total     788
n_unique    788
dtype: int64

,pubs,actual_pub


huffpost                  1
cnn                      10
investing.com             4
politico                  1
time                      0
cnbc                      6
fox                       8
bbc                       7
businessinsider          31
morningstar               0
wsj                       1
nytimes                   0
guardian                  8
reuters                 198
washingtontimes          13
washingtonpost            2
cbs                       1
marketwatch               3
atlantic                  1
vice                      0
npr                       0
newrepublic               0
yahoo                    42
independent              20
heritage                  0
zdnet                     1
townhall                  0
abcnews                   7
hotair                    0
cbc                      15
                       ... 
metro                    36
msnbc                     2
nationalreview            0
news24                   33
techcrunch          

In [27]:
crawler.urls_selected['thinkprogress']

set()

# 3. 해당 URL의 뉴스를 다운로드한다

In [28]:
download_results = crawler.download(); download_results

100.00% (236.16 seconds): Downloading... reuters             

,downloaded,trashed
abcnews,7,0
afr,13,0
aljazeera,8,23
americanconservative,0,0
arirang,0,0
arstechnica,1,0
arynews,4,11
atlantic,1,0
axios,0,0
bbc,5,2


In [20]:
import hashlib
[hashlib.sha1(url.encode('utf-8')).hexdigest() for url in crawler.urls_final['japan-news']['trashed']]

['d80494a4761ead4276e510dc4f64762e319902e0',
 'f25b1d06e58c05786ad57fe9bea9817859de3861',
 '02d668060004e25c2299c9a23445e38a9c6ec771',
 '2d3e330325b35d2c622806feac055478a4a61559',
 '2e87c621d1ffb7f28485496eb82814891ff5dd1b',
 '40cf0a5a2b87e040cd211da1855cb615702cb0ca',
 'feefa98b143ce76ef3cadf0a2764ca634f773fee',
 '0a900188f145b48aad8c70bde741aea563dba542',
 'ef7e33fa30c8be4b6d90681b383234ee4f73e51a',
 '64cc9a5e5a51520306154fda70d482b57a7420b0',
 '12e43a97340dc5a9f9270b38a9705e461456041b',
 'd71b4a23216b50deb276d3ccd8c1905939f5a9c2',
 '9a5e5f813637acd5359da986e562fa9ad9858f4b',
 '44c36bff1fe4d64fb468131f1221959fa30718d6',
 'dbf61f3dd462f9c650d0b2f42ca69717701fc7d1',
 '385dbc7ad54410ac2078055044e34c6e30cfde4b',
 'a31fe291585e194540805022e45df9ebdbabc9fc',
 '66afae226b2dc5d329fc731f7931f9f6d6f37e23']